<div align="right">Python 3.6 Jupyter Notebook</div>

# Sources of data

#### Notebook objectives
By the end of this notebook you will be expected to:
> 
  - Use "trusted" and "untrusted" data sources to enrich your analysis;
  and
  - Understand the implications of the five Rs (**Range**, **Recency**, **Robustness**, **Relevancy**, **Relaibility**) on data quality from external sources.
    
####  List of exercises
>   - **Exercise 1**: Enriching analysis with data from "trusted" sources.
  - **Exercise 2**: Pros and cons of using data from "untrusted" sources.

# Notebook introduction

Data collection is expensive and time consuming, as Arek Stopczynski alluded to in this module's video content.
In some cases, you will be lucky enough to have existing datasets available to support your analysis. You may have datasets from previous analyses, access to data providers, or curated datasets from your organization. In many cases, however, you will not have access to the data that you require to support your analysis, and you will have to find alternate mechanisms. 
The data quality requirements will differ based on the problem you are trying to solve. Taking the hypothetical case of geocoding a location, the accuracy of the geocoded location does not need to be exact when you are simply trying to plot the locations of students on a map. Geocoding a location for an automated vehicle to turn off the highway, on the other hand, has an entirely different accuracy requirement.

> **Note**:

> Those of you who work in large organizations may be privileged enough to have company data governance and data quality initiatives. These efforts and teams can often add significant value both in terms of supplying company-standard curated data, and making you aware of the internal policies that need to be adhered to.

As a **data analyst** or **data scientist**, it is important to be aware of the implications of your decisions. You need to choose the appropriate set of tools and methods to deal with sourcing and supplying data.

Technology has matured in recent years, and allowed access to a host of sources of data that can be used in your analyses. In many cases you can access free resources, or obtain (at a cost) data that has been curated, is at a lower latency, or comes with a service-level agreement. Some governments have even made datasets publicly available.

You have been introduced to [OpenPDS](http://openpds.media.mit.edu/), in the video content, where the focus shifts from supplying raw data -- where the provider needs to apply security principles before sharing datasets -- to supplying answers rather than data. OpenPDS allows users to collect, store, and control access to their data, while also allowing them to protect their privacy. In this way, users still have ownership of their data, as defined by the new deal on data. 

This notebook demonstrates another example of how to source external data to enrich your analyses. The Python ecosystem contains a rich set of tools and libraries that can help you to exploit the available resources.

This course will not go into detail regarding the various options to source and interact with social data from sources such as Twitter, LinkedIn, Facebook, and Google Plus. However, you should be able to find libraries that will assist you in sourcing and manipulating these sources of data.

Twitter data is a good example because, depending on the options selected by the Twitter user, every tweet contains not just the message or content that most users are aware of. It also contains a view of the network of the person, home location, location from which the message was sent, and a number of other features that can be very useful when studying networks around a topic of interest. Professor Alex Pentland pointed out the difference in what you share with the world (how you want to be seen) compared to what you actually do and believe (what you commit to).  Be sure to keep these concepts in mind when you start exploring the additional sources of data. Those who are interested in the topic can start to explore the options by visiting the [Twitter library on PyPI](https://pypi.python.org/pypi/twitter). 

Start with the **five Rs** introduced in Module 1, and consider the following questions:
- How accurate does my dataset need to be?
- How often should the dataset be updated?
- What happens if the data provider is no longer available?
- Do I need to adhere to any organizational standards to ensure consistent reporting or integration with other applications?
- Are there any implications to getting the values wrong?

You may need to start with “untrusted” data sources as a means of validating that your analysis can be executed. Once this is done, you can replace the untrusted components with trusted and curated datasets, as your analysis matures.

<div class="alert alert-warning">
<b>Note</b>:<br>
It is strongly recommended that you save and checkpoint after applying significant changes or completing exercises. This allows you to return the notebook to a previous state should you wish to do so. On the Jupyter menu, select "File", then "Save and Checkpoint" from the dropdown menu that appears.
</div>

#### Load libraries and set options

In [11]:
import pandas as pd
from pandas_datareader import data, wb
import numpy as np
import matplotlib.pylab as plt
import matplotlib
import folium
import geocoder
import wikipedia

#set plot options
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10, 8)

# 1. Source additional data from public sources 
This section will provide short examples to demonstrate the use of public data sources in your notebooks.

## 1.1 World Bank

This example demonstrates how to source data from an external source to enrich your existing analyses. You will need to combine the data sources and add additional features to the example of student locations plotted on the world map in Module 1's Notebook 3.

The specific indicator chosen has little relevance other than to demonstrate the process that you will typically follow in completing your projects. Population counts, from an untrusted source, will be added to your map, and you will use scaling factors combined with the number of students, and population size of the country to demonstrate adding external data with minimal effort.

This example makes use of the [pandas-datareader](https://pandas-datareader.readthedocs.io/en/latest/) module, which supports remote data access. This library has support for extracting data from various internet sources into a Pandas DataFrame. Currently, the supported sources are:

- Yahoo! Finance
- Google Finance
- Enigma
- Quandl
- St.Louis FED (FRED)
- Kenneth French’s data library
- World Bank
- OECD
- Eurostat
- Thrift Savings Plan
- Nasdaq Trader symbol definitions.

This example focuses on enriching your student dataset from Module 1, using the [World Bank's Development Indicators](https://pandas-datareader.readthedocs.io/en/latest/remote_data.html#world-bank). In the following sections, you will use the data you saved in a previous exercise, add corresponding indicators for each country in the data, and find the mean location for all observed coordinates per country.

#### Prepare the student data
In the next code cell, you will load the data from disk, apply the `groupby` method to group the data by country and, for each group, find the total student count and the average of their GPS coordinates. The final dataset containing the country, student count, and averaged GPS coordinates is saved as a separate DataFrame variable. 

In [12]:
import numpy as np
# Load the grouped_geocoded dataset.
df1 = pd.read_csv('data/grouped_geocoded.csv',index_col=[0])
df1.head()

,country,city,industry,student_count,geocode_input,lat,long
0,Australia,Adelaide,Other or Unknown,1,"Adelaide, Australia",-34.928499,138.600746
1,Australia,Bronte,Other or Unknown,1,"Bronte, Australia",-33.905320,151.260690
2,Australia,Buddina,ICT,1,"Buddina, Australia",-26.690000,153.134000
3,Australia,Carlingford,ICT,1,"Carlingford, Australia",-33.777000,151.048000
4,Australia,Glen Waverley,ICT,1,"Glen Waverley, Australia",-37.885652,145.165179


In [13]:
# Prepare the student location dataset for use in this example.
# We use the geometrical center by obtaining the mean location for all observed coordinates per country.
df2 = df1.groupby('country').agg({'student_count': [np.sum], 'lat': [np.mean], 
                                  'long': [np.mean]}).reset_index()
df2.head()

,country,student_count,lat,long
,,sum,mean,mean
0,Australia,26,-34.256125,147.042406
1,Austria,3,47.796835,16.027805
2,Belgium,6,50.807547,4.378567
3,Brazil,24,-21.556434,-44.808458
4,Canada,61,46.172621,-88.355898


In [14]:
# Reset the index.
df3 = df2.reset_index(level=0, drop=True)

# Review the data
df3.head()

,country,student_count,lat,long
,,sum,mean,mean
0,Australia,26,-34.256125,147.042406
1,Austria,3,47.796835,16.027805
2,Belgium,6,50.807547,4.378567
3,Brazil,24,-21.556434,-44.808458
4,Canada,61,46.172621,-88.355898


The column label index has multiple levels. Although this is useful metadata, it would be better to drop multilevel labeling and, instead, rename the columns to capture this information.

In [15]:
# Drop the second level of the columns
df3.columns = df3.columns.droplevel(1)
df3.rename(columns={'lat': "lat_mean", 
                    'long': "long_mean"}, inplace=True)
df3.head()

,country,student_count,lat_mean,long_mean
0,Australia,26,-34.256125,147.042406
1,Austria,3,47.796835,16.027805
2,Belgium,6,50.807547,4.378567
3,Brazil,24,-21.556434,-44.808458
4,Canada,61,46.172621,-88.355898


#### Get and prepare the external dataset from the World Bank
> Remember you can use "``wb.download?``" (without the quotation marks) in a separate code cell to get help on the pandas-datareader method for remote data access of the World Bank Indicators. Refer to the pandas-datareader [remote data access documentation](https://pandas-datareader.readthedocs.io/en/latest/remote_data.html#world-bank) for more detailed help. 

In [16]:
# Access the docstring or inline documentation of the download method from the wb object.
# After running this cell you can close the help by clicking on close (`X`) button in the upper right corner
wb.download?

In [17]:
# The selected indicator is the world population, "SP.POP.TOTL", for the years from 2008 to 2016 
wb_indicator = 'SP.POP.TOTL'
start_year = 2008
end_year = 2016

df4 = wb.download(indicator = wb_indicator,
                  country = ['all'],
                  start = start_year,
                  end = end_year)

In [18]:
# Review the data
df4.tail()

SP.POP.TOTL
country  year             
Zimbabwe 2012   13265331.0
         2011   13025785.0
         2010   12839771.0
         2009   12679810.0
         2008   12550347.0

The data set contains entries for multiple years. The focus of this example is the entry corresponding to the latest year of data available for each country.

In [19]:
df5 = df4.reset_index()
# Create a boolean index to be used to filter the pandas DataFrame df5 based on a specific condition.
# Group df5 by 'country' column and then, within each group, find the maximum value in the 'year' column.
# The 'transform' function returns a Series that has the same length as the original DataFrame df4, 
# with each value replaced by the maximum 'year' value for the corresponding 'country'.
# Compare the maximum 'year' value computed for each 'country' with the 'year' value in each row of DataFrame idx
# The result is a boolean Series where each value is True if the 'year' value in that row is the maximum for
# its 'country', and False otherwise.
idx = df5.groupby(['country'])['year'].transform(max) == df5['year']
idx.head(10)

0     True
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9     True
Name: year, dtype: bool

In [20]:
df5.head()

,country,year,SP.POP.TOTL
0,Africa Eastern and Southern,2016,616377331.0
1,Africa Eastern and Southern,2015,600008150.0
2,Africa Eastern and Southern,2014,583650827.0
3,Africa Eastern and Southern,2013,567891875.0
4,Africa Eastern and Southern,2012,552530654.0


So, '**idx**' is a boolean Series that you can use to filter '**df5**' to only include rows where the 'year' value is the maximum for each 'country'. You can use it like this: '**df6 = df5[idx]**'.

This kind of operation is common when you want to filter a DataFrame to include only the most recent data for each group. In this case, it's finding the most recent year for each country.

You can now extract only the values that correspond to the most recent year available for each country.

In [21]:
# Create a new DataFrame where entries corresponds to maximum year indexes in previous list.
# Use DataFrame df5 '.loc[]' accessor to select rows from 'df5' based on the condition specified by 
# the boolean array 'idx', and assign the resulting DataFrame to 'df6'.
# The ':' inside the brackets '[:]' specifies that all columns are to be included in the operation. 
# So, 'df5.loc[idx, :]' returns a new DataFrame that only includes the rows where the corresponding 
# value in 'idx' is 'True', and includes all columns.
df6 = df5.loc[idx,:]

# Review the data
df6.head()

,country,year,SP.POP.TOTL
0,Africa Eastern and Southern,2016,616377331.0
9,Africa Western and Central,2016,419778384.0
18,Arab World,2016,415077960.0
27,Caribbean small states,2016,7265272.0
36,Central Europe and the Baltics,2016,102994278.0


Now merge your dataset with the World Bank data.

In [22]:
# Combine the student and population datasets.
df7 = pd.merge(df3, df6, on='country', how='left')
df7.head()

,country,student_count,lat_mean,long_mean,year,SP.POP.TOTL
0,Australia,26,-34.256125,147.042406,2016,24190907.0
1,Austria,3,47.796835,16.027805,2016,8736668.0
2,Belgium,6,50.807547,4.378567,2016,11331422.0
3,Brazil,24,-21.556434,-44.808458,2016,206859578.0
4,Canada,61,46.172621,-88.355898,2016,36109487.0


In [23]:
# Rename the columns of our merged dataset and assign to a new variable.
df8 = df7.rename(index=str, columns={('SP.POP.TOTL'): "PopulationTotal_Latest_WB"})
df8.head()

,country,student_count,lat_mean,long_mean,year,PopulationTotal_Latest_WB
0,Australia,26,-34.256125,147.042406,2016,24190907.0
1,Austria,3,47.796835,16.027805,2016,8736668.0
2,Belgium,6,50.807547,4.378567,2016,11331422.0
3,Brazil,24,-21.556434,-44.808458,2016,206859578.0
4,Canada,61,46.172621,-88.355898,2016,36109487.0


In [24]:
# Drop NAN values.
df8 = df8[~df8.PopulationTotal_Latest_WB.isnull()]

# Reset index.
df8.reset_index(level=0, drop=True)

,country,student_count,lat_mean,long_mean,year,PopulationTotal_Latest_WB
0,Australia,26,-34.256125,147.042406,2016,2.419091e+07
1,Austria,3,47.796835,16.027805,2016,8.736668e+06
2,Belgium,6,50.807547,4.378567,2016,1.133142e+07
3,Brazil,24,-21.556434,-44.808458,2016,2.068596e+08
4,Canada,61,46.172621,-88.355898,2016,3.610949e+07
5,Chile,1,-33.448890,-70.669265,2016,1.808388e+07
6,China,6,25.726065,113.358120,2016,1.387790e+09
7,Costa Rica,2,9.920695,-84.146152,2016,4.945205e+06
8,Croatia,2,45.810939,15.896201,2016,4.174349e+06
9,Denmark,2,55.676097,12.568337,2016,5.728010e+06


Let's plot the data.
> **Note**:
The visualization below does not have any meaning. The scaling factors selected are used to demonstrate the difference in population sizes, and number of students on this course, per country.

In [25]:
# Plot the combined dataset

# Set map center and zoom level
mapc = [0, 30]
zoom = 2

# Create map object.
map_osm = folium.Map(location=mapc,
                   tiles='OpenStreetMap',
                    zoom_start=zoom)

# Plot each of the locations that we geocoded.
for j in range(len(df8)):
    # Plot a blue circle marker for country population.
    folium.CircleMarker([df8.lat_mean[j], df8.long_mean[j]],
                    radius=df8.PopulationTotal_Latest_WB[j]/20000000,
                    popup='Population',
                    color='#3186cc',
                    fill_color='#3186cc',
                   ).add_to(map_osm)
    # Plot a red circle marker for students per country.
    folium.CircleMarker([df8.lat_mean[j], df8.long_mean[j]],
                    radius=df8.student_count[j]/50,
                    popup='Students',
                    color='red',
                    fill_color='red',
                   ).add_to(map_osm)
# Show the map.
map_osm

<br>
<div class="alert alert-info">
<b>Exercise 1 Start.</b>
</div>

### Instructions

> 1. Review the available indicators in the [World Bank](http://data.worldbank.org/indicator) dataset, and select an indicator of your choice (other than the population indicator).  
2. Using a **copy** of the code (from above) in the cells below, replace the population indicator with your selected indicator. Instead of returning the most recent value for your selected indicator, compute the mean and standard deviation for the years from 2006 to 2016. You will need to use the Pandas ``groupby().agg()`` chained methods, together with the following functions from NumPy: 
  - ``np.mean``
  - ``np.std``.
  
> You can review the data preparation section for the student data above for an example. 

>Add comments (lines starting with a "#") giving a brief description of your view on the observed results. Make sure to include, in one or two sentences in each case, the following:
   1. A clear description why you selected the indicator.
   - What your expectation was before including the data.
   - What you think the results may indicate.

> **Important**:
- Only the external data needs to be prepared. You do not need to prepare the student dataset again. Just use the student data that you prepared above and join this to the new dataset you sourced. 
- Only plot the mean values for your selected indicator (**not** the standard deviation values).


In [26]:
# Load world bank data set
wb.download?

# My World Bank indicator: Expenditure on education as % of total government expenditure (%) 'SE.XPD.TOTL.GB.ZS'

# Indicator selected is a of proxy how countries prioritize education. 

wb_indicator = 'SE.XPD.TOTL.GB.ZS'
start_year = 2006
end_year = 2016

df9 = wb.download(indicator = wb_indicator,
                  country = ['all'],
                  start = start_year,
                  end = end_year)
df9.head(20)

SE.XPD.TOTL.GB.ZS
country                     year                   
Africa Eastern and Southern 2016          17.198811
                            2015          16.962910
                            2014          18.097099
                            2013          17.243259
                            2012          17.315451
                            2011          17.022470
                            2010          17.548059
                            2009                NaN
                            2008          16.258141
                            2007                NaN
                            2006          18.004990
Africa Western and Central  2016          12.854880
                            2015          13.121995
                            2014          12.939880
                            2013          14.963605
                            2012          14.332880
                            2011          14.563540
                            2010          17.057570
                            2009          18.175619
                            2008          18.031691

In [27]:
# Rename column of the dataset and assign to a new variable "EducExp_TotGovExp_WB". 

df10 = df9.rename(index=str, columns={('SE.XPD.TOTL.GB.ZS'): "EducExp_TotGovExp_WB"}).reset_index()

# Drop NAN values.

df10 = df10[~df10.EducExp_TotGovExp_WB.isnull()]

df10.head()

,country,year,EducExp_TotGovExp_WB
0,Africa Eastern and Southern,2016,17.198811
1,Africa Eastern and Southern,2015,16.962910
2,Africa Eastern and Southern,2014,18.097099
3,Africa Eastern and Southern,2013,17.243259
4,Africa Eastern and Southern,2012,17.315451


In [28]:
# Computing the mean and standard deviation of "EducExp_TotGovExp_WB" by country. 

df11 = df10.groupby('country')['EducExp_TotGovExp_WB'].agg([np.mean, np.std]).reset_index()
df11.head()

,country,mean,std
0,Afghanistan,14.645883,1.860139
1,Africa Eastern and Southern,17.294577,0.557166
2,Africa Western and Central,15.636861,2.232537
3,Albania,12.187144,1.051917
4,Algeria,15.311616,1.996494


In [29]:
# Rename columns 'mean' and 'std'

df12 = df11.rename(columns={('mean'): "EducExp_TotGovExp_WB_Mean", 'std':'EducExp_TotGovExp_WB_Std'})

# Reset the index.

#df13 = df12.reset_index()

df12.head()

,country,EducExp_TotGovExp_WB_Mean,EducExp_TotGovExp_WB_Std
0,Afghanistan,14.645883,1.860139
1,Africa Eastern and Southern,17.294577,0.557166
2,Africa Western and Central,15.636861,2.232537
3,Albania,12.187144,1.051917
4,Algeria,15.311616,1.996494


In [30]:
# Combine the student with mean_EXP and and std_EXP datasets.

df13 = pd.merge(df3, df12, on='country', how='left')

df13.head()

# My expectation was that countries would spend significantly higher than the returned mean on eduction.
# This dataset indicates spending is very low.

,country,student_count,lat_mean,long_mean,EducExp_TotGovExp_WB_Mean,EducExp_TotGovExp_WB_Std
0,Australia,26,-34.256125,147.042406,13.548056,0.526959
1,Austria,3,47.796835,16.027805,10.657574,0.204337
2,Belgium,6,50.807547,4.378567,11.864937,0.384273
3,Brazil,24,-21.556434,-44.808458,14.799902,1.202489
4,Canada,61,46.172621,-88.355898,12.197085,0.426226


In [31]:
# Plot the combined dataset

# Set map center and zoom level
mapc = [0, 30]
zoom = 2

# Create map object.
map_osm = folium.Map(location=mapc,
                   tiles='OpenStreetMap',
                    zoom_start=zoom)

# Plot each of the geocoded locations.
for j in range(len(df13)):
    # Plot a blue circle marker for country Mean.
    folium.CircleMarker([df13.lat_mean[j], df13.long_mean[j]],
                    radius=df13.EducExp_TotGovExp_WB_Mean[j],
                    popup='Mean',
                    color='#3186cc',
                    fill_color='#3186cc',
                   ).add_to(map_osm)
    # Plot a red circle marker for students per country.
    folium.CircleMarker([df13.lat_mean[j], df13.long_mean[j]],
                    radius=df13.student_count[j]/50,
                    popup='Students',
                    color='red',
                    fill_color='red',
                   ).add_to(map_osm)
# Show the map.
map_osm

<br>
<div class="alert alert-info">
<b>Exercise 1 End.</b>
</div>

> **Exercise complete**:
    
> This is a good time to "Save and Checkpoint".

## 1.2 Using Wikipedia as a data source

To demonstrate how quickly data can be sourced from public, "untrusted" data sources, you have been supplied with a number of sample scripts below. While these sources contain distinctly rich datasets, which you can acquire with minimal effort, they can be amended by anyone, and may not be 100% accurate. In some cases, you will have to manually transform the datasets, while in others, you might be able to use pre-built libraries.

Execute the code cells below before completing Exercise 2.

# Display MIT page summary from Wikipedia 

# Set the language of wikipedia API to English (optional, this is the default)
wikipedia.set_lang("en")

# Get the summary of the MIT page
# The 'summary()' function from the wikipedia module in Python retrieves the summary section of the Wikipedia 
# page for the topic given as its argument.
# If you want to retrieve the full text of the page, use the 'page()' function instead.
mit_summary = wikipedia.summary("Massachusetts Institute of Technology")

# Print the summary
print(mit_summary)

In [ ]:
# Display a single sentence summary.
wikipedia.summary("Massachusetts Institute of Technology", sentences = 1)

In [ ]:
# Create variable page that contains the wikipedia information.
page = wikipedia.page("List of countries and dependencies by population")

# Display the page title.
page.title

In [1]:
# Display the page URL. This can be utilised to create links back to descriptions.
page.url

NameError: name 'page' is not defined

<br>
<div class="alert alert-info">
<b>Exercise 2 Start.</b>
</div>

### Instructions

> After executing the cells for the Wikipedia example in Section 1.2, think about the potential implications of using this "public" and, in many cases, "untrusted" data source when doing analysis or creating data products.

> **Please compile and submit for evaluation a list of *three pros* and *three cons*.**

>> **Note**: Your submission can be a simple markdown list using the syntax provided below.

Add your answer in this markdown cell. The contents of this cell should be replaced with your answer.

**Submit as a list:**
>**Pros**:

    - 
    - 
    - 
>**Cons**:

    - 
    - 
    - 

<br>
<div class="alert alert-info">
<b>Exercise 2 End.</b>
</div>

> **Exercise complete**:
    
> This is a good time to "Save and Checkpoint".